In [1]:
#| default_exp fmf_biobert

# Found My Fitness Example
>"Here we'll try using elements from the biobertApi module to label biomedical terms in the comments section of a Found My Fitness Youtube video"

In [2]:
#|hide
# import biobert-bern 
from biobert_bern.biobertApi import *

Less than 5k characters. Only 1 block necessary.


In [3]:
# !pip install pathlib

In [4]:
import requests
import re
import pandas as pd
from nbdev.showdoc import *
import fastai
from pathlib import *
import numpy as np     

In [5]:
path = Path('../..')
comment_csv= 'found_my_fitness_UCWF8SqJVNlx-ctXbLswcTcA_youtube_comments_only_122rows.csv'

In [6]:
coms = pd.read_csv(path/comment_csv)

In [7]:
coms.rename(columns = {'comment': 'comments'},inplace=True)

In [8]:
coms.tail(2)

,vid_id,vid_channel_id,vid_deets,vid_response,title,publishedAt,tcomment_id,tcresponse,textDisplay,textOriginal,totalReplyCount,rep_comment,reply_id,comments
120,g4QqJoox8tc,NaN,NaN,NaN,NaN,2022-11-02T11:20:13Z,NaN,"{'kind': 'youtube#comment', 'etag': 'yd5KSzb3_...",... and they said it gives you the munchies.,... and they said it gives you the munchies.,NaN,"{'kind': 'youtube#comment', 'etag': 'yd5KSzb3_...",Ugy2ZO2t370rDO-eWoZ4AaABAg.9husbe2WJcN9hvjM9rBbfr,... and they said it gives you the munchies.
121,g4QqJoox8tc,NaN,NaN,NaN,NaN,2022-11-02T03:07:55Z,Ugx3lPLxH7NyOIZxzEl4AaABAg,"{'kind': 'youtube#commentThread', 'etag': 'Gzv...",My first thought was Rich Roll,My first thought was Rich Roll,0.0,NaN,NaN,My first thought was Rich Roll


## Now start splitting off all the cumsums <5000 and stack them back together.

### Create Section Labels

In [9]:
coms = create_com_with_idx(coms)

# Create Text Block
> Here we'll put all the text together

In [10]:
# coms = coms[['textDisplay']]
# coms.rename(columns={'textDisplay':'comments'},inplace=True)

In [11]:
# df =coms
# df = make_5k_sections(df)
# blocks = []
# for o in df.section.unique():
#     dfbl = df.loc[df.section == o]
#     text_block = ' '.join(dfbl.comidx.tolist())
#     blocks.append(text_block)

In [15]:

# text_block = create_text_block(df=coms);# text_block
blocks = create_all_text_blocks(df=coms)

/notebooks/git/biobertapi1/biobert-bern/biobert_bern/biobertApi.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dffk['section'] = counter
/notebooks/git/biobertapi1/biobert-bern/biobert_bern/biobertApi.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cumTxtLen'] = df.textLen.cumsum()


In [16]:
type(blocks)

list

In [17]:
for o in blocks: print(o[:50]); print()

0:: if exercise and/or the interaction with alcoho

28:: that's hilarious. my alcoholic triathlete bro

43:: as one of the most severe alcoholics ever bor

60:: i really really really appreciate this concis

77:: as a recovering alcoholic, damn sign me up fo

99:: i have to force myself, both to do exercise a



In [ ]:
# text_block[:500]

### Get Comment Spans

In [ ]:
# dfi = get_comment_spans_textblock(text_block); dfi.tail(3)

In [ ]:
len(text_block)

In [ ]:
# # Send it all to Bern2
# output = query_plain(text=text_block);  

## Try by comment

In [ ]:
df = pd.DataFrame()
for o in blocks.text_block[:2]:
    print( len(o))
#     out = query_plain(text=o)
#     df = df.append({'out':out},ignore_index=True)
# df.tail()

In [ ]:
df.out[0]['annotations']

In [ ]:
df2 = pd.DataFrame()
for p,o in zip(df.index, df.out):    
    dft = pd.DataFrame(o['annotations'])
    dft['comidx'] = p
    df2 = df2.append(dft)
df2

In [ ]:
df2

In [ ]:
pd.DataFrame(df['annotations'])